In [ ]:

# STACKING ENSEMBLE WITH TABNET AS META MODEL


# 1️⃣ Import Libraries

import numpy as np
import pandas as pd
import joblib
import os
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from pytorch_tabnet.tab_model import TabNetClassifier
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler


In [3]:
# 2️⃣ Load Dataset (Final Preprocessed CSV)

df = pd.read_csv("D:/FYP/data/datasets/final_dataset_02.csv")

# Separate features and labels
X = df.drop(columns=['label'])
y = df['label'].astype(int)


In [5]:
# 3️⃣ Load Base Models' Predictions


# base models were trained and saved in .pkl format in the trained_models folder
base_model_paths = {
    "xgboost": "D:/FYP/models/trained_models/xgboost_native_model.pkl",
    #"catboost": "models/trained_models/catboost_intrusion_detection.pkl",
    "lightgbm": "D:/FYP/models/trained_models//lightgbm_intrusion_detection.pkl"
}

# Load trained base models
xgb_model = joblib.load(base_model_paths["xgboost"])
#cat_model = joblib.load(base_model_paths["catboost"])
lgb_model = joblib.load(base_model_paths["lightgbm"])


In [ ]:
# 4️⃣ Generate Base Model Predictions (as Meta Features)

# These predictions become input features for TabNet (meta model)

# Split dataset
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

# Get base model predictions (probabilities recommended for stacking)
xgb_train_preds = xgb_model.predict_proba(X_train)[:, 1]
xgb_test_preds = xgb_model.predict_proba(X_test)[:, 1]

# cat_train_preds = cat_model.predict_proba(X_train)[:, 1]
# cat_test_preds = cat_model.predict_proba(X_test)[:, 1]

lgb_train_preds = lgb_model.predict_proba(X_train)[:, 1]
lgb_test_preds = lgb_model.predict_proba(X_test)[:, 1]

# Create new meta-feature dataset for TabNet  will uncomment if catboost issue solved
# X_meta_train = np.vstack((xgb_train_preds, cat_train_preds, lgb_train_preds)).T
# X_meta_test = np.vstack((xgb_test_preds, cat_test_preds, lgb_test_preds)).T
X_meta_train = np.vstack((xgb_train_preds,  lgb_train_preds)).T
X_meta_test = np.vstack((xgb_test_preds,  lgb_test_preds)).T

AttributeError: 'Booster' object has no attribute 'predict_proba'

In [ ]:
# 5️⃣ Normalize Meta Features

scaler = StandardScaler()
X_meta_train_scaled = scaler.fit_transform(X_meta_train)
X_meta_test_scaled = scaler.transform(X_meta_test)


In [ ]:
# 6️⃣ Initialize TabNet Classifier

tabnet_model = TabNetClassifier(
    n_d=8, n_a=8,
    n_steps=3,
    gamma=1.5,
    lambda_sparse=1e-4,
    optimizer_fn=torch.optim.Adam,
    optimizer_params=dict(lr=2e-2),
    mask_type='sparsemax',
    verbose=10,
    seed=42
)


In [ ]:
# 7️⃣ Train the TabNet Meta Model

tabnet_model.fit(
    X_meta_train_scaled, y_train.values,
    eval_set=[(X_meta_test_scaled, y_test.values)],
    eval_name=["valid"],
    eval_metric=["accuracy"],
    max_epochs=100,
    patience=20,
    batch_size=512,
    virtual_batch_size=128,
    num_workers=0
)


In [ ]:
# 8️⃣ Meta Model Predictions & Evaluation

y_meta_pred = tabnet_model.predict(X_meta_test_scaled)

# Evaluation
accuracy = accuracy_score(y_test, y_meta_pred)
print(f"\n✅ Meta Model Accuracy: {accuracy:.4f}")
print("\n✅ Classification Report:\n", classification_report(y_test, y_meta_pred))

# Confusion Matrix
conf_matrix = confusion_matrix(y_test, y_meta_pred)
plt.figure(figsize=(6,4))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=['Normal', 'Attack'], yticklabels=['Normal', 'Attack'])
plt.title("TabNet Meta Model Confusion Matrix")
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.tight_layout()
plt.show()


In [ ]:
# 9️⃣ Save the Meta Model

os.makedirs("D:/FYP/models/trained_models", exist_ok=True)
joblib.dump(tabnet_model, "D:/FYP/models/trained_models/tabnet_meta_model.pkl")
print("✅ TabNet Meta Model saved at: models/trained_models/tabnet_meta_model.pkl")

# Save the scaler too (important during inference)
joblib.dump(scaler, "D:/FYP/models/trained_models/meta_scaler.pkl")
print("✅ Meta feature scaler saved at: D:/FYP/models/trained_models/meta_scaler.pkl")
